<img src="https://github.com/chdb-io/chdb/raw/main/docs/_static/snake-chdb.png" width=150>

# chdb + urleng
This example shows the combined power of chdb and urleng to persist sessions on the cloud

* [chdb](https://chdb.io) is an in-memory OLAP database powered by ClickHouse
* [urleng.com](https://urleng.com) is a free pastie service for ClickHouse url tables

In [ ]:
!pip install chdb --upgrade --quiet

In [41]:
import uuid
from chdb import session as chs

## Generate a secret, unique secret urleng.com session token & save for later
## url_token = str(uuid.uuid1())
url_token = 'dc81bb0a-4902-11ee-a84b-0242ac1c000c' # just for demo purposes
%store url_token

print(url_token)

## Create DB, Table, View in temp session, auto cleanup when session is deleted.
sess = chs.Session()

sess.query("CREATE DATABASE IF NOT EXISTS db_xxx ENGINE = Atomic")
sess.query("CREATE TABLE IF NOT EXISTS db_xxx.log_table_xxx (key String, value UInt64) ENGINE = Log;")
sess.query("INSERT INTO db_xxx.log_table_xxx VALUES ('a', 1), ('b', 3), ('c', 2), ('d', 5);")

print("Select from session:\n")
print(sess.query("SELECT * FROM db_xxx.log_table_xxx", "Pretty"))

## Copy to urlengine table, persist the session on the cloud.
sess.query("INSERT INTO FUNCTION url('https://urleng.com/"+url_token+"', JSONEachRow, 'key String, value UInt64') SELECT * FROM db_xxx.log_table_xxx")

Stored 'url_token' (str)
dc81bb0a-4902-11ee-a84b-0242ac1c000c
Select from session:

┏━━━━━┳━━━━━━━┓
┃ key ┃ value ┃
┡━━━━━╇━━━━━━━┩
│ a   │     1 │
├─────┼───────┤
│ b   │     3 │
├─────┼───────┤
│ c   │     2 │
├─────┼───────┤
│ d   │     5 │
└─────┴───────┘



In [42]:
from chdb import session as chs

## Create a new DB, Table, View in temp session, auto cleanup when session is deleted.
new_sess = chs.Session()

## Retrieve the urlengine session token
%store -r url_token
print(url_token)

new_sess.query("CREATE DATABASE IF NOT EXISTS db_xxx ENGINE = Atomic")
new_sess.query("CREATE TABLE IF NOT EXISTS db_xxx.log_table_xxx (key String, value UInt64) ENGINE = Log;")

## There's no data in our session yet. Zero results.
print(new_sess.query("SELECT * FROM db_xxx.log_table_xxx", "Pretty"))

## Restore session from the cloud, insert data from urleng table
new_sess.query("INSERT INTO db_xxx.log_table_xxx SELECT * FROM url('https://urleng.com//"+url_token+"', JSONEachRow)")

print("Select from restored session:\n")
print(new_sess.query("SELECT * FROM db_xxx.log_table_xxx", "Pretty"))


dc81bb0a-4902-11ee-a84b-0242ac1c000c

Select from restored view:

┏━━━━━┳━━━━━━━┓
┃ key ┃ value ┃
┡━━━━━╇━━━━━━━┩
│ a   │     1 │
├─────┼───────┤
│ b   │     3 │
├─────┼───────┤
│ c   │     2 │
├─────┼───────┤
│ d   │     5 │
└─────┴───────┘

